<a href="https://colab.research.google.com/github/NihPat95/Instacart-Market-Analysis/blob/master/Instacart_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import time

colors = sns.color_palette()

In [28]:
!pip install efficient_apriori

In [3]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/Instacart/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
orders = pd.read_pickle(path + 'customers_orders10000.pkl')

In [8]:
print(orders.shape)

(6140, 2)


In [0]:
x = orders['items'].values

In [37]:
from efficient_apriori import apriori
%time itemsets, rules = apriori(x, min_support=0.001,  min_confidence=0.01)

CPU times: user 14.2 s, sys: 0 ns, total: 14.2 s
Wall time: 14.2 s


In [38]:
itemsets

{1: {(10027,): 7,
  (10060,): 179,
  (10101,): 7,
  (10130,): 28,
  (10148,): 14,
  (10195,): 8,
  (10360,): 19,
  (10365,): 56,
  (10375,): 7,
  (10386,): 30,
  (10388,): 94,
  (10405,): 7,
  (10421,): 78,
  (10424,): 35,
  (10455,): 20,
  (10518,): 185,
  (10521,): 92,
  (10557,): 14,
  (10603,): 379,
  (10872,): 99,
  (10978,): 119,
  (10990,): 55,
  (10998,): 155,
  (11029,): 44,
  (11066,): 10,
  (11110,): 33,
  (11165,): 11,
  (11223,): 11,
  (11253,): 17,
  (11279,): 13,
  (11362,): 23,
  (11381,): 33,
  (11451,): 13,
  (11585,): 10,
  (11610,): 99,
  (11703,): 13,
  (11747,): 7,
  (11798,): 98,
  (11843,): 12,
  (11925,): 174,
  (11930,): 59,
  (11945,): 264,
  (12068,): 77,
  (12121,): 83,
  (12179,): 7,
  (12191,): 42,
  (12196,): 62,
  (12315,): 183,
  (12361,): 11,
  (12492,): 169,
  (12495,): 7,
  (12575,): 16,
  (12709,): 19,
  (12738,): 143,
  (12816,): 50,
  (12822,): 94,
  (12834,): 34,
  (12905,): 23,
  (12992,): 13,
  (13034,): 10,
  (13040,): 27,
  (13086,): 7,
  (1

In [39]:
rules

[{10603} -> {10060},
 {10060} -> {10603},
 {14678} -> {10060},
 {10060} -> {14678},
 {17789} -> {10060},
 {10060} -> {17789},
 {26307} -> {10060},
 {10060} -> {26307},
 {26668} -> {10060},
 {10060} -> {26668},
 {29345} -> {10060},
 {10060} -> {29345},
 {10365} -> {10148},
 {10148} -> {10365},
 {17565} -> {10148},
 {10148} -> {17565},
 {10360} -> {14678},
 {14399} -> {10365},
 {10365} -> {14399},
 {17565} -> {10365},
 {10365} -> {17565},
 {22275} -> {10388},
 {10388} -> {22275},
 {30170} -> {10388},
 {10388} -> {30170},
 {40376} -> {10388},
 {10388} -> {40376},
 {30170} -> {10421},
 {10421} -> {30170},
 {40376} -> {10421},
 {10421} -> {40376},
 {14678} -> {10518},
 {10518} -> {14678},
 {19611} -> {10518},
 {10518} -> {19611},
 {26668} -> {10518},
 {10518} -> {26668},
 {34209} -> {10518},
 {10518} -> {34209},
 {38720} -> {10518},
 {10518} -> {38720},
 {22275} -> {10521},
 {10521} -> {22275},
 {40376} -> {10521},
 {10521} -> {40376},
 {10978} -> {10603},
 {10603} -> {10978},
 {11610} -> {